In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

In [24]:
def evaluate(final_embedding, word2int, embed_size):
    gensim = GensimWrapper('data/all.txt',embed_size, 0, log=False)
    gensim.set_embeddings(word2int, final_embedding)
    result = gensim.evaluate()
    for key in result:
        print("{0}: {1:.2f}%".format(key, result[key]), end=' ')
    print()

In [3]:
words = read_file(filename='data/all.txt')
vocab, word2int, int2word = build_vocab(words)
word2freq = get_frequency(words, word2int, int2word)

words = min_count_threshold(words, word2freq, 5)
vocab, word2int, int2word = build_vocab(words)
print(len(vocab))

int_words = words_to_ints(word2int, words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 100
skip_window = 5

179219


In [19]:
def parseVec(file, delimiter):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    embeddings = np.ndarray((vocab_size+1, embed_size-1), dtype=np.float32)
    for i in range(vocab_size):
        line = lines[i+1].split(delimiter)[:-1]
        word = line[0]
        if word in word2int:
            wordvec = np.array([float(j) for j in line[1:]])
            embeddings[word2int[word]] = wordvec
    return embeddings

In [121]:
util = Utils(word2int, int2word, fast)
print(util.evaluate_word_analogy('data/analogies.txt'))

In [20]:
utils = Utils(embedding=fast, word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

NameError: name 'fast' is not defined

In [123]:
evaluate(embed_size=embed_size,final_embedding=normalize(fast), word2int=word2int)
evaluate(embed_size=embed_size,final_embedding=fast, word2int=word2int)

c:\users\amany\appdata\local\conda\conda\envs\gputf3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


anomaly: 73.39% semantic: 0.54% syntactic: 16.67% 
anomaly: 56.88% semantic: 2.70% syntactic: 15.15% 


In [124]:
utils = Utils(embedding=cew, word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

[('ኢትዮጵያ', 515.95514),
 ('አገሪቱ', 325.75748),
 ('አፍሪካ', 275.1266),
 ('አገራችን', 231.23654),
 ('አሜሪካ', 212.9898),
 ('ኢትዮጵያን', 208.88614),
 ('ኤርትራ', 206.08847),
 ('በኢትዮጵያ', 200.81245),
 ('ኢትዮጵያም', 193.45535),
 ('ግብፅ', 189.0678)]

In [21]:
cew = parseVec('results/cc.am.300.vec', ' ')

In [22]:
evaluate(embed_size=cew.shape[1],final_embedding=normalize(cew), word2int=word2int)
evaluate(embed_size=cew.shape[1],final_embedding=cew, word2int=word2int)

D:\Library\Projects\amharic_word_embedding\data_handle.py:260: RuntimeWarning: invalid value encountered in true_divide
  return array / norms
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


anomaly: 71.56% semantic: 0.95% syntactic: 0.00% 
anomaly: 41.28% semantic: 2.69% syntactic: 0.00% 
